In [1]:
## Import
import pandas as pd
from pandas.plotting import autocorrelation_plot
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from scipy import stats

In [2]:
import os
main_data_path = r"C:\Users\erkay\B2Metric_Case\home-credit-default-risk"
dataset_path = os.path.join(main_data_path,"application_train.csv")
df = pd.read_csv(dataset_path)

In [3]:
pos_cash_dataset_path = os.path.join(main_data_path,"POS_CASH_balance.csv")
df_pos_cash           = pd.read_csv(pos_cash_dataset_path)

In [4]:
df_pos_cash[df_pos_cash["SK_ID_CURR"]==226561]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
404156,1932377,226561,-3,12.0,11.0,Active,0,0
1698703,1932377,226561,-2,12.0,10.0,Active,0,0
3204398,1932377,226561,-4,12.0,12.0,Active,0,0
6642100,2118158,226561,-6,12.0,7.0,Active,0,0
6644564,2118158,226561,-8,12.0,9.0,Active,0,0
7074605,2118158,226561,-5,12.0,6.0,Active,0,0
7074861,2118158,226561,-9,12.0,10.0,Active,0,0
7256558,1932377,226561,-1,12.0,9.0,Active,0,0
7351151,2118158,226561,-2,12.0,3.0,Active,0,0
7355131,2118158,226561,-3,12.0,4.0,Active,0,0


In [5]:
pd.set_option('display.max_columns', None)

In [6]:
#print(df_pos_cash.columns[6:8])
pos_column_names = df_pos_cash.columns
pos_column_names[4]

'CNT_INSTALMENT_FUTURE'

In [7]:
print(df_pos_cash["SK_ID_CURR"].nunique())

337252


In [8]:
unique_pos_ids = df_pos_cash["SK_ID_CURR"].unique()

df_pos_unique = pd.DataFrame(columns=(df_pos_cash.columns[1:]))
temp_index = 0
for unique_id in unique_pos_ids:
    if temp_index%1000==0:
        print(temp_index)
    temp_df = df_pos_cash[df_pos_cash["SK_ID_CURR"]==unique_id]
    temp_status = temp_df["NAME_CONTRACT_STATUS"].mode()
    temp_df_amt1 = []
    for column_name1 in pos_column_names[2:4]:
        temp_df_amt1.append(temp_df[column_name1].mean(axis=0))
    temp_df_amt2 = []
    for column_name2 in [pos_column_names[4]]:
        new_temp = temp_df[column_name2]
        temp_df_amt2.append(new_temp.sum())
    temp_df_amt3 = []
    for column_name3 in pos_column_names[6:8]:
        temp_df_amt3.append(temp_df[column_name3].mean(axis=0))
    
    temp_all = [unique_id] + temp_df_amt1 + temp_df_amt2 + [temp_status[0]] + temp_df_amt3
    # append rows to an empty DataFrame
    df_pos_unique.loc[temp_index,df_pos_cash.columns[1:]] = temp_all
    temp_index = temp_index + 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000


KeyboardInterrupt: 

In [46]:
df_pos_unique

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,226558,-24.092308,13.538462,535.0,Active,185.584615,0.0
1,226561,-5.066667,12.0,119.0,Active,0.0,0.0


In [49]:
col_new = []
for col in df_pos_unique.columns:
    if col != "SK_ID_CURR":
        col_new.append("POS_"+col)
    else:
        col_new.append("SK_ID_CURR")

In [50]:
df_pos_unique.columns = col_new

In [51]:
df_pos_unique

,SK_ID_CURR,POS_MONTHS_BALANCE,POS_CNT_INSTALMENT,POS_CNT_INSTALMENT_FUTURE,POS_NAME_CONTRACT_STATUS,POS_SK_DPD,POS_SK_DPD_DEF
0,182943,-27.186047,38.930233,1092.0,Active,0.0,0.0
1,367990,-41.148148,26.444444,516.0,Active,0.0,0.0
2,397406,-42.449541,18.238532,1280.0,Active,37.706422,0.146789
3,269225,-45.859649,32.850877,2473.0,Active,0.0,0.0
4,334279,-37.126984,25.0,1824.0,Active,0.0,0.0
...,...,...,...,...,...,...,...
1580,322573,-41.444444,21.066667,1275.0,Active,0.022222,0.022222
1581,315250,-26.166667,22.541667,1223.0,Active,0.097222,0.097222
1582,388908,-52.25,13.478261,639.0,Active,0.0,0.0
1583,422816,-28.216495,19.515464,1364.0,Active,0.371134,0.371134
